In [2]:
!pip install qiskit_optimization 

> Define the ILP formulation of the BPP. You can use Docplex or similar frameworks to do it. 

Given $n$ items, each with an associated weight $w_i$, and bins with maximum weight (capacity) $C$. $x_{ij}$ represent decision variables that equals 1 if item $i$ is put into bin $j$, and $y_j$ = 1 if bin $j$ is used, bounded by the maximum number of bins $n$. The objective is to minimize the number of bins
$$
\quad \sum_{j=1}^{n} y_j
$$

subject to constraints that 
1. 1 item is assigned to 1 bin
$$
\quad \sum_{j=1}^{n} x_{ij} = 1, \quad \forall i = 1, \ldots, n
$$
2. total weight of items respect the bin capacity
$$
\quad \sum_{i=1}^{n} w_i x_{ij} \leq C, \quad \forall j = 1, \ldots, n
$$
3. if a bin is not used $y_j = 0$, no items should be assigned to it, so $x_{ij} = 0$ for all i. 
$$
\quad x_{ij} \leq y_j, \quad \forall i = 1, \ldots, n, \forall j = 1, \ldots, n
$$
We can also modify the right hand side of capacity constraint from C to $C y_j$, which could implies the last constraint, but it is separated to improve the performance of the model.


In [3]:
from docplex.mp.model import Model

def define_ilp(items_weight: list, bin_capacity: float, verbose = False) -> Model:
    """
    Define an integer linear programming model for the bin packing problem.

    Parameters
    ----------
    items_weight : list of float
        Weights of the items.
    bin_capacity : float
        naximum weight that a bin can hold.
    """
    N = len(items_weight)
    model = Model()
    x = {(i, j): model.binary_var() for i in range(N) for j in range(N)}
    y = [model.binary_var() for j in range(N)]
    model.minimize(model.sum(y[j] for j in range(N)))
    for i in range(N):
        model.add_constraint(model.sum(x[i, j] for j in range(N)) == 1)
    for j in range(N):
        model.add_constraint(model.sum(items_weight[i] * x[i, j] for i in range(N)) <= bin_capacity)
    for i in range(N):
        for j in range(N):
            model.add_constraint(x[i, j] <= y[j])
    if verbose:
        print(f"Number of variables: {model.number_of_variables} and constraints: {model.number_of_constraints}")
        print(f"Model objective: {model.get_objective_expr()}")
    return model


> Create a function to transform the ILP model into a QUBO 

The objective function is quatradic with the form $x^T Q x$, where x is a vector of binary decision variables and Q is a square matrix of constants. There is no constraints so we introduce quadratic penalties into the objective function as an alternative. 
In general, converting a equality constraint $a x = b$ to a penalty term $(a x - b)^2$ takes the form of
- constant $ C += b**2 $
- linear $ L[x_i] -= 2 coef[x_i] * b $
- quadratic $ Q[x_i, x_j] += coef[x_i] * coef[x_i] $

A inequality constraint can be converted to equality form by including slack variables in a binary expansion. The number of slack variables per equation is $log_2(b)$. The equation becomes $(a x + 2^l s_l = b)$, where $s$ is the slack variable with $2^l$ as coefficient. 

The constraint $x_i \leq x_j$ is special can be converted to $x_i - x_j x_i$.

The matrix Q is constructed as the first n rows and columns are the $x_{ij}$ variables, and the last n rows and columns are the $y_j$ variables. For a small system with 2 items with weights 1 and 2 and 2 bins with capacity 3, assuming penalty factor is 1, the objective function is
$$
Q = x5 + x6
$$
1. Item Assignment Constraint:
$$
x1+x2 == 1, x3 + x4 == 1
$$
$$
P_1 = (1 - x1 - x2)^2 = 1 - 2 x1 - 2 x2 + x1^2 + x2^2 + 2 x1 x2 = 1 - x1 - x2 + 2 x1 x2
$$

2. Bin Capacity Constraint:
$$
x1 + 2 x3 <= 3 x5,
x2 + 2 x4 <= 3 x6
$$
P_3 = (3x5 - x1 - 2 x3)^2 = 9 x5^2 - 6 x1 x5 - 12 x3 x5 + x1^2 + 4 x1 x3 + 4 x3^2
P_4 = (3x6 - x2 - 2 x4)^2 = 9 x6^2 - 6 x2 x6 - 12 x4 x6 + x2^2 + 4 x2 x4 + 4 x4^2
$$
we only penalize when A is positive (i.e., when the left-hand side exceeds the right-hand side). For QUBO, this can be approximated with squared penalties

3. Bin Assignment Constraint:
$$
x1 <= x5, x2 <= x6, x3 <= x5, x4 <= x6
$$
P_5 = (x5 - x1)^2 = x5^2 - 2 x1 x5 + x1^2
P_6 = (x6 - x2)^2 = x6^2 - 2 x2 x6 + x2^2
P_7 = (x5 - x3)^2 = x5^2 - 2 x3 x5 + x3^2
P_8 = (x6 - x4)^2 = x6^2 - 2 x4 x6 + x4^2
$$

The QUBO matrix is the sum of the penalties and objective function
$$
Q = x5 + x6 + P_1 + P_2 + P_3 + P_4 + P_5 + P_6 + P_7 + P_8
$$

In general, convert a equality constraint $a x = b$ to a penalty term $(a x - b)^2$ 
constant part += penalty * b**2
linear part, for each variable $x_i$ -= 2 penalty * coef * b
quadratic part, for each pair of variables $x_i$ and $x_j$ += 2 penalty * coef_1 * coef_2

a inequality constraint can always be put in equality form by including slack variables and then representing the slack variables by a binary expansion.
constant part += penalty * b**2
linear part, for each variable $x_i$ += 2 penalty * coef * constant

In [9]:
import numpy as np
import math
def constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, rhs, coeffs: np.ndarray):
    constant = penalty_factor * rhs ** 2
    linear = np.zeros(num_total_variables)
    quadratic = np.zeros((num_total_variables, num_total_variables))
    for item_idx in item_idx_list: # for each bin that item could be placed
        # linear term
        linear[item_idx] -= 2 * penalty_factor * rhs * coeffs[item_idx]
        # quadratic term
        for item_jdx in item_idx_list:
            quadratic[item_idx, item_jdx] += penalty_factor * coeffs[item_idx]* coeffs[item_jdx]
    return constant, linear, quadratic

def define_qubo(items_weight: list, bin_capacity: float, penalty_factor: float = 1.0, verbose = False) -> np.ndarray:
    """
    Define a QUBO model for the bin packing problem.

    Parameters
    ----------
    items_weight : list of float
        Weights of the items.
    bin_capacity : float
        Maximum weight that a bin can hold.
    penalty_factor : float
        Penalty factor for the constraints.
    """
    num_item = len(items_weight)
    item_variables = (0, num_item ** 2)
    bin_variables = (item_variables[1], item_variables[1] + num_item)
    num_slack_variables_per_eq = math.floor(np.log2(bin_capacity))
    last_slack_coeff = None
    if 2 **num_slack_variables_per_eq - 1 < bin_capacity:
        last_slack_coeff = bin_capacity + 1 - 2 ** (num_slack_variables_per_eq)
        num_slack_variables_per_eq += 1
    slack_variables = (bin_variables[1], bin_variables[1] + num_slack_variables_per_eq * num_item)
    num_total_variables = slack_variables[1]
    P2_coeffs = np.zeros(num_total_variables)
    for i in range(num_item):
        for j in range(num_item):
            P2_coeffs[i * num_item + j] = items_weight[i]
            
    for i in range(num_item):
        for l in range(num_slack_variables_per_eq):
            if last_slack_coeff is not None and l == num_slack_variables_per_eq - 1:
                P2_coeffs[slack_variables[0] + i * num_slack_variables_per_eq + l] = last_slack_coeff
            else:
                P2_coeffs[slack_variables[0] + i * num_slack_variables_per_eq + l] = 2 ** l
            
    constant = 0
    linear = np.zeros(num_total_variables)
    quadratic = np.zeros((num_total_variables, num_total_variables))
    # objective: 
    for i in range(bin_variables[0], bin_variables[1]):
        linear[i] = 1
    # constraints:
    # P_1 = (1 - x1 - x2 - ... - xN)^2 = 1 - 2 * (x1 + x2 + ... + xN) + (x1^2 + x2^2 + ... + xN^2)
    for i in range(0, num_item**2, num_item): # for constraints on each item
        item_idx_list = range(i, num_item + i)
        P1_coeffs = np.ones(num_total_variables)
        Pconstant, Plinear, Pquadratic = constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, 1, P1_coeffs)
        constant += Pconstant
        linear = np.add(linear, Plinear)
        quadratic = np.add(quadratic, Pquadratic)
    # P_2 = (x1 + x2 + ... + xN - y1)^2 = x1^2 + x2^2 + ... + xN^2 - 2 * (x1 * y1 + x2 * y2 + ... + xN * yN) + y1^2
    for b in range(0, num_item): # for each bin 
        item_idx_list = range(b, num_item**2, num_item) 
        slack_idx_list = range(slack_variables[0] + b * num_slack_variables_per_eq, slack_variables[0] + (b + 1) * num_slack_variables_per_eq)
        item_idx_list = list(item_idx_list) + list(slack_idx_list)
        Pconstant, Plinear, Pquadratic = constraint_to_penalty(num_total_variables, item_idx_list, penalty_factor, bin_capacity, P2_coeffs)
        constant += Pconstant
        linear = np.add(linear, Plinear)
        quadratic = np.add(quadratic, Pquadratic)
    # P_3 = x1 - x1x5 + x2 - x2x6 + ... 
    for b in range(0, num_item): # for each bin 
        bin_idx = bin_variables[0] + b
        for item_idx in range(b, num_item**2, num_item): # for each item that could be placed in the bin
            linear[item_idx] += penalty_factor
            # quadratic term
            quadratic[item_idx, bin_idx] -= penalty_factor
    # convert quadratic to upper triangular matrix by adding the lower triangular part
    quadratic += quadratic.T - np.diag(quadratic.diagonal())
    quadratic[np.tril_indices(num_total_variables, -1)] = 0
    if verbose:
        print(f"Total number of variables: {num_total_variables}, item idx {item_variables}, bin idx {bin_variables}, slack idx {slack_variables}")
        print(f"Offset: {constant}, num of linear terms: {np.count_nonzero(linear)}, num of quadratic terms: {np.count_nonzero(quadratic)}")
    return constant, linear, quadratic

> Test your function with specific instances (size small, medium, and big) 

In [10]:
import numpy as np
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

np.random.seed(42)

instances = {
    'small': {
        'weights': [1, 2], 
        'bin_capacity': 13
    },
    'medium': {
        'weights': np.random.randint(1, 10, size=8),
        'bin_capacity': 20
    },
    'large': {
        'weights': np.random.randint(1, 50, size=32),
        'bin_capacity': 100                     
    }
}
for instance in ['small', 'medium', 'large']:
    model = define_ilp(instances[instance]['weights'], instances[instance]['bin_capacity'])
    constant, linear, quadratic = define_qubo(instances[instance]['weights'], instances[instance]['bin_capacity'], penalty_factor=3.0, verbose=True)
    test_qp = from_docplex_mp(model)
    conv = QuadraticProgramToQubo(penalty=3.0)
    test_qubo = conv.convert(test_qp)
    test_constant = test_qubo.objective.constant
    test_linear = test_qubo.objective.linear.to_array() 
    test_quadratic = test_qubo.objective.quadratic.to_array()
    if np.isclose(constant, test_constant) and np.allclose(linear, test_linear) and np.allclose(quadratic, test_quadratic):
        print(f"{instance} instance: PASSED")
    else:
        print(f"{instance} instance: FAILED")
        print(f"constant error: {np.abs(constant - test_constant)}")
        print(f"linear error: {np.abs(linear - test_linear)}")
        print(f"quadratic error: {np.abs(quadratic - test_quadratic)}")

Total number of variables: 14, item idx (0, 4), bin idx (4, 6), slack idx (6, 14)
Offset: 1020.0, num of linear terms: 14, num of quadratic terms: 48
small instance: PASSED
Total number of variables: 112, item idx (0, 64), bin idx (64, 72), slack idx (72, 112)
Offset: 9624.0, num of linear terms: 112, num of quadratic terms: 1016
medium instance: PASSED
Total number of variables: 1280, item idx (0, 1024), bin idx (1024, 1056), slack idx (1056, 1280)
Offset: 960096.0, num of linear terms: 1280, num of quadratic terms: 41856
large instance: PASSED


In [6]:
define_qubo(instances['small']['weights'], instances['small']['bin_capacity'])

(340.0,
 array([ -27.,  -27.,  -53.,  -53.,    1.,    1.,  -26.,  -52., -104.,
        -156.,  -26.,  -52., -104., -156.]),
 array([[ 2.,  2.,  4.,  0., -1.,  0.,  2.,  4.,  8., 12.,  0.,  0.,  0.,
          0.],
        [ 0.,  2.,  0.,  4.,  0., -1.,  0.,  0.,  0.,  0.,  2.,  4.,  8.,
         12.],
        [ 0.,  0.,  5.,  2., -1.,  0.,  4.,  8., 16., 24.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  5.,  0., -1.,  0.,  0.,  0.,  0.,  4.,  8., 16.,
         24.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  4.,  8., 12.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4., 16., 24.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16., 48.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 36.,  0.,  0.,  0.,
 

> Create a Brute Force solver for the QUBO problem and solve the specific instances. 

1. Generate all $2^n$ possible solutions; in practice, we can limit the search space by the constraints.
2. Evaluate the objective function for each combination
3. Track the best solution

In [7]:
def qubo_brute_force(Q, items_weight = []):
    from itertools import product
    n = Q.shape[0]
    if len(items_weight) > 0:
        assert n == len(items_weight) * 2
    min_energy = float('inf')
    min_state = None
    for state in product([0, 1], repeat=n):
        energy = np.dot(state, np.dot(Q, state))
        if energy < min_energy:
            min_energy = energy
            min_state = state
    return min_state, min_energy
model = define_ilp(instances['small']['weights'], instances['small']['bin_capacity'])
qubo = ilp_to_qubo(model, penalty_factor=1.0)
qubo_brute_force(qubo)

NameError: name 'ilp_to_qubo' is not defined

> To solve the QUBO, use quantum annealing simulators. You can use the Dwave Ocean Framework.

In [6]:
# !pip install dwave-ocean-sdk
# !dwave config create # config file created in github codespace


In [11]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

def quantum_annealing_qubo(Q, sampler):
    qubo_dict = {(i, j): Q[i, j] for i in range(Q.shape[0]) for j in range(Q.shape[1]) if Q[i, j] != 0}
    # Submit the QUBO to the quantum annealer
    sampleset = sampler.sample_qubo(qubo_dict, num_reads=100)
    best_solution = sampleset.first.sample
    best_energy = sampleset.first.energy

    return best_solution, best_energy

sampler = EmbeddingComposite(DWaveSampler())
Q_example = np.array([[1, -1, 0],
                      [-1, 2, -1],
                      [0, -1, 2]])
best_solution, best_energy = quantum_annealing_qubo(Q_example, sampler)

print("Best solution:", best_solution)
print("Best energy:", best_energy)

Best solution: {0: np.int8(0), 1: np.int8(0), 2: np.int8(0)}
Best energy: 0.0


> Use a Quantum Variational approach to solve the QUBO. 
Create multiple Ansantz for tests. 
Build a function with input being the QUBO and Ansantz. Using a hybrid approach solved the QUBO. 

define a cost (or loss) function C which encodes the solution to the problem
proposes an ansatz, that is, a quantum operation depending on a set of continuous or discrete parameters θ that can be optimizedtrained in a hybrid quantum-classical loop to solve the optimization task

In [ ]:
from qiskit_optimization.algorithms import Doc

> Use QAOA to solve the QUBO. 

Create from scratch a QAOA function. 
as p increases the approximation improves.
1. preparing qubits in an equal superposition over all possible states by applying a Hadamard gate to each qubit
2. in each run of circuit:
    1. in each layer:
        1. apply the problem Hamiltonian to evolve the quantum state, making states with higher costs less favorable
        2. apply the mixer Hamiltonian to explore the solution space. The Pauli-X operator is applied to flip the qubits, helping the algorithm to explore different bitstrings (configurations).
        3. parameters γ and β are optimized to minimize the cost function.
    2. measure the quantum state to get a candidate solution

the Problem Hamiltonian 𝐻 encodes the problem constraints and objective 
$$
H_p = \sum_{i=1}^{n} (1 - \sum_{j=1}^{n} x_{ij})^2 + \sum_{j=1}^{n}  \max (0, (C y_j - \sum_{i=1}^{n} w_i x_{ij})^2) + \sum_{i=1}^{n} \sum_{j=1}^{n} x_{ij} y_j
$$


In [1]:
# Define the problem: Example bin packing problem cost function (simplified)
def bin_packing_cost(bitstring):
    # A simple example cost function where the goal is to minimize the number of "bins" used
    # Here, the bitstring encodes whether an item is placed in a bin.
    # You can modify this function to represent the actual bin packing constraints and penalties.
    bins_used = sum(int(bit) for bit in bitstring)  # Number of bins used
    return bins_used  # Example of a cost function to minimize the bins used

# Create the QAOA circuit
def create_qaoa_circuit(n_qubits, p, gamma, beta):
    """
    Create a QAOA circuit with p layers.
    
    Parameters:
    - n_qubits: Number of qubits (items/bins).
    - p: Number of QAOA layers.
    - gamma: List of gamma parameters.
    - beta: List of beta parameters.
    """
    # Create quantum circuit
    qc = QuantumCircuit(n_qubits)
    
    # Step 1: Initialize in superposition (Hadamard on all qubits)
    for qubit in range(n_qubits):
        qc.h(qubit)

    # Step 2: Apply the QAOA layers
    for layer in range(p):
        # Apply the cost Hamiltonian
        for qubit in range(n_qubits):
            qc.rz(2 * gamma[layer], qubit)  # Phase rotation to encode cost

        # Apply the mixer Hamiltonian (Pauli-X rotations)
        for qubit in range(n_qubits):
            qc.rx(2 * beta[layer], qubit)  # X rotation to mix states

    # Step 3: Measure the qubits
    qc.measure_all()

    return qc

# Define the cost function to minimize during classical optimization
def qaoa_cost_function(params, n_qubits, p, backend, shots=1024):
    """
    The function to optimize. Runs the QAOA circuit and calculates the cost.
    
    Parameters:
    - params: A list of parameters (alternating gamma and beta).
    - n_qubits: Number of qubits.
    - p: Number of layers.
    - backend: The quantum backend to execute the circuit on.
    - shots: The number of shots (runs) for each circuit execution.
    """
    gamma = params[:p]
    beta = params[p:]
    
    # Create the QAOA circuit with the given parameters
    qc = create_qaoa_circuit(n_qubits, p, gamma, beta)
    
    # Transpile and execute the circuit
    transpiled_qc = transpile(qc, backend)
    job = execute(transpiled_qc, backend, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)
    
    # Calculate the cost as the weighted sum of the outcomes
    total_cost = 0
    for bitstring, count in counts.items():
        cost = bin_packing_cost(bitstring)  # Calculate the cost for each outcome
        total_cost += cost * count / shots  # Weighted by the number of times each outcome occurred
    
    return total_cost

# Run QAOA
def run_qaoa(n_qubits, p, backend, shots=1024):
    """
    Run the QAOA optimization.
    
    Parameters:
    - n_qubits: Number of qubits (bins/items in bin packing).
    - p: Number of layers for the QAOA.
    - backend: The quantum backend to execute the circuit on.
    - shots: The number of shots (circuit executions).
    """
    # Initial guesses for gamma and beta parameters
    initial_params = np.random.rand(2 * p)

    # Classical optimization using scipy.optimize.minimize
    result = minimize(qaoa_cost_function, initial_params, args=(n_qubits, p, backend, shots),
                      method='COBYLA', options={'maxiter': 200})
    
    # Get the optimized parameters
    optimized_params = result.x
    gamma_opt = optimized_params[:p]
    beta_opt = optimized_params[p:]
    
    # Return the optimal gamma and beta parameters
    return gamma_opt, beta_opt

In [2]:
n_qubits = 4  # Number of items/bins (for bin packing)
p = 2  # Number of QAOA layers

# Use the Qiskit Aer simulator as the backend
backend = Aer.get_backend('qasm_simulator')

# Run QAOA to optimize parameters
gamma_opt, beta_opt = run_qaoa(n_qubits, p, backend)

# Print optimized parameters
print("Optimized gamma:", gamma_opt)
print("Optimized beta:", beta_opt)

# Create the final QAOA circuit with the optimized parameters
final_qc = create_qaoa_circuit(n_qubits, p, gamma_opt, beta_opt)

# Execute the final circuit to get the solution
transpiled_qc = transpile(final_qc, backend)
job = execute(transpiled_qc, backend, shots=1024)
result = job.result()

# Plot the histogram of results
counts = result.get_counts(final_qc)
print("Final measurement outcomes:", counts)
plot_histogram(counts)

NameError: name 'Aer' is not defined

Compare and analyze the results. 

What is the difference between QAOA, Quantum Annealing, and Quantum Variational approaches with different Ansatz? 
The QUBO model has emerged as an underpinning of the quantum computing area known as quantum annealing and Fujitsu's digital annealing, and has become a subject of study in neuromorphic computing.

QVE variational optimization of a quantum circuit to minimize the expectation value of a given Hamiltonian. The optimization is performed iteratively, with the quantum circuit parameters updated at each step until the most optimal solution is determined. 

QAOA, on the other hand, is a quantum algorithm that prepares a quantum state that is a superposition of all possible solutions to the problem. The algorithm applies a sequence of unitary operations to the initial state, with the number of operations and their parameters being determined by the problem the QAOA algorithm has been designed to solve.

VQE is better suited for problems that require a high degree of precision, while QAOA is better suited for problems with a large array of initial possibilities. VQE is known to be more efficient than QAOA for problems with a lower variety of initial solutions, as it can converge to the optimal solution faster. However, for problems with a higher variety of initial solutions, VQE may become computationally expensive due to the exponential growth of the required resources. QAOA, on the other hand, is designed to handle problems with a large search space efficiently.

How do the results compare with the brute force approach? 

## References
1. Quantum Bridge Analytics I: A Tutorial on Formulating and Using QUBO Models https://arxiv.org/pdf/1811.11538
2. A Quantum Approximate Optimization Algorithm https://arxiv.org/pdf/1411.4028
3. Variational quantum algorithms https://arxiv.org/pdf/2012.09265
4. Quantum and quantum-inspired optimization for solving the minimum bin packing problem https://arxiv.org/pdf/2301.11265
4. https://github.com/qiskit-community/qiskit-optimization/blob/807d48167caf11cd93bec85f26b34614fb7868da/docs/tutorials/02_converters_for_quadratic_programs.ipynb#L16
Comparing VQE and QAOA: Two Quantum Algorithms for Optimization Problems https://www.quantumgrad.com/article/700
https://qiskit-community.github.io/qiskit-optimization/tutorials/01_quadratic_program.html